# Basic EM workflow 3 (Restaurants data set)

# Introduction

This IPython notebook explains a basic workflow two tables using *py_entitymatching*. Our goal is to come up with a workflow to match restaurants from Fodors and Zagat sites. Specifically, we want to maximize F1. The datasets contain information about the restaurants.

First, we need to import *py_entitymatching* package and other libraries as follows:

In [1]:
import sys
import py_entitymatching as em
import pandas as pd
import os

In [2]:
# Display the versions
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )

python version: 3.6.5 (default, Apr  1 2018, 05:46:30) 
[GCC 7.3.0]
pandas version: 0.22.0
magellan version: 0.3.0


# Read input tables

We begin by loading the input tables. For the purpose of this guide, we use the datasets that are included with the package.

In [3]:
# Get the paths
path_A = 'data/amazon_products.csv'
path_B = 'data/walmart_products.csv'

In [4]:
# Load csv files as dataframes and set the key attribute in the dataframe
A = em.read_csv_metadata(path_A, key='id')
B = em.read_csv_metadata(path_B, key='id')

In [5]:
print('Number of tuples in A    : ' + str(len(A)))
print('Number of tuples in B    : ' + str(len(B)))
print('Number of tuples in A X B: ' + str(len(A)*len(B)))

Number of tuples in A    : 2976
Number of tuples in B    : 4847
Number of tuples in A X B: 14424672


In [6]:
A.head(3)

,id,product title,brand,model,operating system,combo
0,a0,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,acer,nh.q28aa.001,windows 10,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...
1,a1,acer aspire e 15 15.6 full hd 8th gen intel core i5-8250u geforce mx150 8gb ram memory 256gb ssd...,acer,e5-576g-5762,windows 10,acer aspire e 15 15.6 full hd 8th gen intel core i5-8250u geforce mx150 8gb ram memory 256gb ssd...
2,a2,asus vivobook f510ua fhd laptop intel core i5-8250u 8gb ram 1tb hdd usb-c nanoedge display finge...,asus,f510ua-ah51,windows 10,asus vivobook f510ua fhd laptop intel core i5-8250u 8gb ram 1tb hdd usb-c nanoedge display finge...


In [7]:
B.head(3)

,id,product title,brand,model,operating system,combo
0,w0,iview i896qw 8.95 2-in-1 32gb tablet intel atom bay trail z3735f processor windows 10,iview,NaN,microsoft windows,iview i896qw 8.95 2-in-1 32gb tablet intel atom bay trail z3735f processor windows 10 iview micr...
1,w1,dell - inspiron 2-in-1 11.6 touch-screen laptop - intel pentium - 4gb memory - 500gb hd - red,dell,i3168-3270red,windows 10,dell - inspiron 2-in-1 11.6 touch-screen laptop - intel pentium - 4gb memory - 500gb hd - red de...
2,w2,iview maximus ii 11.6 laptop touchscreen 2-in-1 windows 10 intel bay trail z3735f processor 2gb ...,iview,max2-bk,windows 10,iview maximus ii 11.6 laptop touchscreen 2-in-1 windows 10 intel bay trail z3735f processor 2gb ...


In [8]:
# Display the keys of the input tables
em.get_key(A), em.get_key(B)

('id', 'id')

# Block tables to get candidate set

Before we do the matching, we would like to remove the obviously non-matching tuple pairs from the input tables. This would reduce the number of tuple pairs considered for matching.
*py_entitymatching* provides four different blockers: (1) attribute equivalence, (2) overlap, (3) rule-based, and (4) black-box. The user can mix and match these blockers to form a blocking sequence applied to input tables.

For the matching problem at hand, we know that two restaurants with different city names will not match. So we decide the apply blocking over names:

In [9]:
# Blocking plan

# A, B -- attribute equiv. blocker [model] --------------------|---> candidate set

In [10]:
# Create overlap blocker
ob = em.OverlapBlocker()

# Use block_tables to apply blocking over two input tables.
C1 = ob.block_tables(A,B, 'model', 'model', 
                    l_output_attrs=['id','model','combo'], 
                    r_output_attrs=['id','model','combo'],
                    overlap_size=5,
                    q_val=5,
                    word_level=False,
                    show_progress=False,
                    n_jobs=-1
                    )
len(C1)

22427

In [11]:
C1.head(3)

,_id,ltable_id,rtable_id,ltable_model,ltable_combo,rtable_model,rtable_combo
0,0,a229,w1,i3168-3271blu,dell i3168-3271blu 11.6 hd 2-in-1 laptop (intel pentium n3710 1.6ghz processor 4 gb ddr3l sdram ...,i3168-3270red,dell - inspiron 2-in-1 11.6 touch-screen laptop - intel pentium - 4gb memory - 500gb hd - red de...
1,1,a1812,w1,dell inspiron i3168,high performance dell inspiron 11.6 touchscreen 2 in 1 laptop pc intel pentium n3710 quad-core p...,i3168-3270red,dell - inspiron 2-in-1 11.6 touch-screen laptop - intel pentium - 4gb memory - 500gb hd - red de...
2,2,a2596,w1,i3542-5666red,dell inspiron 3000 series 15.6 non-touch notebook (red) intel core i3-4005u 4gb memory 500 gb ha...,i3168-3270red,dell - inspiron 2-in-1 11.6 touch-screen laptop - intel pentium - 4gb memory - 500gb hd - red de...


## Debug blocker output

The number of tuple pairs considered for matching is reduced to 10165 (from 176423), but we would want to make sure that the blocker did not drop any potential matches. We could debug the blocker output in *py_entitymatching* as follows:

In [12]:
# Debug blocker output
dbg = em.debug_blocker(C1, A, B, output_size=10, 
                       attr_corres=[
                           ('product title','product title'),
                           ('brand', 'brand'), 
                           ('model','model'),
                           ('combo','combo')],
                      verbose=True)
#### Display first few tuple pairs from the debug_blocker's output
dbg.head(3)

,_id,ltable_id,rtable_id,ltable_product title,ltable_brand,ltable_model,ltable_combo,rtable_product title,rtable_brand,rtable_model,rtable_combo
0,0,a1116,w434,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...,hp,NaN,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...,hp,hp-15-02395-fhd-me28,hp envy x360 15z premium yoga style 2-in-1 convertible laptop (amd ryzen 5 quad core apu radeon ...
1,1,a1341,w2999,asus zenbook ux430ua-dh74 thin and light ultrabook 14 fhd laptop pc (intel 8th gen i7 quad core ...,hp,NaN,asus zenbook ux430ua-dh74 thin and light ultrabook 14 fhd laptop pc (intel 8th gen i7 quad core ...,asus zenbook ux430ua-dh74 thin and light ultrabook 14 fhd laptop pc (intel 8th gen i7 quad core ...,hp,asu-14-07054-fhd-me4,asus zenbook ux430ua-dh74 thin and light ultrabook 14 fhd laptop pc (intel 8th gen i7 quad core ...
2,2,a28,w2192,asus fx503vm 15.6” fhd powerful gaming laptop intel core i7-7700hq quad-core 2.8ghz (turbo up to...,asus,fx503vm-eh73,asus fx503vm 15.6” fhd powerful gaming laptop intel core i7-7700hq quad-core 2.8ghz (turbo up to...,asus fx503vm 15.6” fhd intel core i7-7700hq quad-core 2.8ghz (turbo up to 3.8ghz) 3gb gtx 1060 1...,asus,90nr0gp1-m00060,asus fx503vm 15.6” fhd intel core i7-7700hq quad-core 2.8ghz (turbo up to 3.8ghz) 3gb gtx 1060 1...


From the debug blocker's output we observe that the current blocker drops quite a few potential matches. We would want to update the blocking sequence to avoid dropping these potential matches.

For the considered dataset, we know that for the restaurants to match the  names must overlap between them. We could use overlap blocker for this purpose. Finally, we would want to union the outputs from the attribute equivalence blocker and the overlap blocker to get a consolidated candidate set.

In [13]:
# Updated blocking sequence
# A, B ------ overlap blocker [combo] --------> C1--
#                                                   |----> C
# A, B ------ overlap blocker [model] --------> C2--

In [14]:
# Create overlap blocker
ob = em.OverlapBlocker()

# Block tables using 'combo' attribute 
C2 = ob.block_tables(A, B, 'combo', 'combo', 
                    l_output_attrs=['id','combo'], 
                    r_output_attrs=['id','combo'],
                    overlap_size=50,
                    q_val=5,
                    word_level=False,
                    show_progress=False,
                    n_jobs=-1
                    )
len(C2)

64314

In [15]:
# Display first two rows from C2
C2.head()

,_id,ltable_id,rtable_id,ltable_combo,rtable_combo
0,0,a1765,w1,dell inspiron i3000-10099slv 11.6 inch touchscreen 2-in-1 laptop (intel pentium 4 gb ram 500 gb ...,dell - inspiron 2-in-1 11.6 touch-screen laptop - intel pentium - 4gb memory - 500gb hd - red de...
1,1,a2842,w1,dell - inspiron 2-in-1 15.6 touch-screen laptop (intel core i5 8gb memory 500gb hard drive black...,dell - inspiron 2-in-1 11.6 touch-screen laptop - intel pentium - 4gb memory - 500gb hd - red de...
2,2,a233,w4,black flip design lenovo 11.6-inch touchscreen 2-in-1 business laptop intel celeron n3060 4gb me...,dell inspiron 11 3168 11.6 laptop touchscreen 2-in-1 windows 10 home intel celeron n3060 process...
3,3,a236,w4,lenovo 11.6?? ips touchscreen 2-in-1 convertible laptop pc intel celeron processor up to 2.48ghz...,dell inspiron 11 3168 11.6 laptop touchscreen 2-in-1 windows 10 home intel celeron n3060 process...
4,4,a255,w4,asus vivobook flip tp200sa-dh01t 11.6 inch display thin and lightweight 2-in-1 hd touchscreen la...,dell inspiron 11 3168 11.6 laptop touchscreen 2-in-1 windows 10 home intel celeron n3060 process...


We add a rule-based blocker.

In [18]:
brands = ['gigabyte','lg', 'alienware', 'microsoft', 'huawei', 'rca', 'razer', 'iview', 'toshiba',
          'hp', 'dell', 'lenovo', 'prostar', 'acer', 'samsung', 'apple', 'asus', 'panasonic', 'msi']
models = ['transformer','precision','460','470','560','570','p50','p51','p70','p71','aero','sabre','miix','titan','gram','gl','aw', 'aspire', 'swift', 'spin', 'strix', 'raider', 'cambio', 'nitro', 'matebook', 'viking', 'probook', 'phantom', 'helios', 'triton', 'stream', 'blade', 'zenbook', 'zephyrus', 'flex', 'notebook', 'colorwheel', 'surface', 'chromebook',
          'vivobook', 'maximus', 'zenbook', 'flex', 'ativ', 'predator', 'x360', 'stealth', 'omen', 'xps', 'carbon', 'x1', 'yoga', 'envy', 'thinkpad', 'latitude', 'inspiron', 'elitebook', 'clevo', 'spectre', 'macbook', 'pavilion', 'ideapad', 'legion']

def match_make_model(a, b):
    # if for all value sets, there is a value that is in both tuples, return false (keep tuple).
    # otherwise return true (drop tuple).
    a_combo = a['combo']
    b_combo = b['combo']
    valuesets = [brands, models]
    i = 0
    n = len(valuesets)
    while i < n:
        valueset = valuesets[i]
        j = 0
        m = len(valueset)
        match = False
        while j < m:
            value = valueset[j]
            if value in a_combo and value in b_combo:
                # end loop
                j = m
                # mark as match
                match = True
            j += 1
        if not match:
            # drop tuple
            return True
        i += 1
    return False

# 30x slower oneliner:
#     return not all(
#         [any([value in a['combo'] and value in b['combo']] for value in valueset) 
#          for valueset in [brands, models, cpus]])

In [ ]:
# for i in range(len(A)):
#     if match_make_model(A.iloc[i],A.iloc[i]):
#         print(A.iloc[i]['combo'])
#         print()

In [20]:
bb = em.BlackBoxBlocker()
bb.set_black_box_function(match_make_model)
C3 = bb.block_tables(A, B,
                    l_output_attrs=['combo'], 
                    r_output_attrs=['combo'],
                    show_progress=True,
                    n_jobs=-1)
len(C3)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:03:41


565184

In [21]:
C3 = C4
C3.head(3)

,_id,ltable_id,rtable_id,ltable_combo,rtable_combo
0,0,a0,w769,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,manufacturer refurbished acer predator 17 g9-792-790g 17.3 laptop touchscreen windows 10 home in...
1,1,a1,w83,acer aspire e 15 15.6 full hd 8th gen intel core i5-8250u geforce mx150 8gb ram memory 256gb ssd...,manufacturer refurbished acer aspire r5-471t-71lx 14 laptop touchscreen 2-in-1 windows 10 home i...
2,2,a1,w112,acer aspire e 15 15.6 full hd 8th gen intel core i5-8250u geforce mx150 8gb ram memory 256gb ssd...,manufacturer refurbished acer aspire r5-471t-79yn 14 laptop touchscreen 2-in-1 windows 10 home i...


In [22]:
# Combine blocker outputs
C = em.combine_blocker_outputs_via_union([C1,C2,C3])
len(C)

605460

In [23]:
C.head()

,_id,ltable_id,rtable_id,ltable_model,ltable_combo,rtable_model,rtable_combo
0,0,a0,w1111,nh.q28aa.001,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,nx.vg0aa.001,acer travelmate spin b1 b118-rn-c6fd - 11.6 - celeron n3450 - 4 gb ram - 64 gb ssd - us internat...
1,1,a0,w1115,nh.q28aa.001,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,nt.lcwaa.001,windows 10 pro education (64-bit) intel atom x5-z8350 2mb cache 1.44ghz up to acer nt.lcwaa.001 ...
2,2,a0,w1116,nh.q28aa.001,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,nx.gmbaa.001,refurbished acer spin laptop intel pentium 1.10 ghz 4 gb ram 64gb windows 10 home acer nx.gmbaa....
3,3,a0,w1221,nh.q28aa.001,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,nt.lceaa.001,acer aspire switch alpha 12 sa5-271p-38uz 12 touchscreen led [in-plane switching [ips] technolog...
4,4,a0,w1355,nh.q28aa.001,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,nh.q2qaa.006,acer nitro 5 gaming laptop 15.6 fhd screen intel core i7-7700hq geforce gtx 1050ti graphic card ...


We observe that the number of tuple pairs considered for matching is increased to 12530 (from 10165). Now let us debug the blocker output again to check if the current blocker sequence is dropping any potential matches.

In [ ]:
# Debug again
dbg = em.debug_blocker(C, A, B, output_size=10)
dbg.head()

We observe that the current blocker sequence does not drop obvious potential matches, and we can proceed with the matching step now. A subtle point to note here is, debugging blocker output practically provides a stopping criteria for modifying the blocker sequence.


# Matching tuple pairs in the candidate set

In this step, we would want to match the tuple pairs in the candidate set. Specifically, we use learning-based method for matching purposes.
This typically involves the following five steps:
1. Sampling and labeling the candidate set
2. Splitting the labeled data into development and evaluation set
3. Selecting the best learning based matcher using the development set
4. Evaluating the selected matcher using the evaluation set

## Sampling and labeling the candidate set

First, we randomly sample 450 tuple pairs for labeling purposes.

In [ ]:
# Sample  candidate set
# S = em.sample_table(C, 500)
# em.to_csv_metadata(S, 'data/new.csv')
# S.head()

For the purposes of this guide, we will load in a pre-labeled dataset (of 450 tuple pairs) included in this package.

In [ ]:
# # Load the pre-labeled data
path_G = 'data/labeled.csv'
G = em.read_csv_metadata(path_G, 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')
len(G)

## Splitting the labeled data into development and evaluation set

In this step, we split the labeled data into two sets: development (I) and evaluation (J). Specifically, the development set is used to come up with the best learning-based matcher and the evaluation set used to evaluate the selected matcher on unseen data.

In [ ]:
# Split S into development set (I) and evaluation set (J)
IJ = em.split_train_test(G, train_proportion=0.7, random_state=0)
I = IJ['train']
J = IJ['test']

In [ ]:
G.head()


## Selecting the best learning-based matcher 

Selecting the best learning-based matcher typically involves the following steps:

1. Creating a set of learning-based matchers
2. Creating features
3. Converting the development set into feature vectors
4. Selecting the best learning-based matcher using k-fold cross validation

### Creating a set of learning-based matchers

In [ ]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

### Creating features

Next, we need to create a set of features for the development set. *py_entitymatching* provides a way to automatically generate features based on the attributes in the input tables. For the purposes of this guide, we use the automatically generated features.

In [ ]:
match_t = em.get_tokenizers_for_matching([2,3,4,5,6,7,8,9])
match_s = em.get_sim_funs_for_matching()
atypes1 = em.get_attr_types(A)
atypes2 = em.get_attr_types(B)
match_c = em.get_attr_corres(A, B)
feature_table = em.get_features(A, B, atypes1, atypes2, match_c, match_t, match_s)

This function generates a function that returns true if both or neither tuples' attribute contain any of the passed in values and false otherwise.

In [ ]:
def generateContainsValueFeature(values, name=None, attribute='combo'):
    if type(values) is str:
        values = [values]
    def containsValueFeature(a,b):
        return int(any([value.lower() in a[attribute].lower() for value in values]) 
                   == any([value.lower() in b[attribute].lower() for value in values]))
    return containsValueFeature, name if name else values[0]

Use this to generate many new features.

In [ ]:
brands = ['toshiba','hp','dell','lenovo','prostar','acer','samsung','apple','asus','panasonic','msi']
models = ['zenbook','flex','ativ','predator','x360','stealth','omen','xps','carbon','x1','yoga','envy','thinkpad','latitude','inspiron','elitebook','clevo','spectre','macbook','pavilion','ideapad','legion']
thinkpads = [['p50','p51','p71'],['460','470','560','570']]
asus = ['swift','aspire','spin']
sizes = ['13','15','17']
operating_systems = ['chrome','windows','mac']
cpus = ['i3','i5','i7','celeron','pentium']
miscellaneous = ['2-in-1','gtx','touch']
keywords = brands + models + thinkpads + sizes + operating_systems + cpus + miscellaneous
new_features = [generateContainsValueFeature(value) for value in keywords]

for feature in new_features:
    em.add_blackbox_feature(feature_table, feature[1], feature[0])

In [ ]:
# List the names of the features generated
feature_table['feature_name']

### Converting the development set to feature vectors

In [ ]:
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=feature_table, 
                            attrs_after='match',
                            show_progress=False)
H.fillna(0, inplace=True)
H.head()

### Selecting the best matcher using cross-validation

Now, we select the best matcher using k-fold cross-validation. For the purposes of this guide, we use five fold cross validation and use the 'precision' metric to select the best matcher.

In [ ]:
# Select the best ML matcher using CV
result = em.select_matcher(
        matchers=[dt, rf, svm, ln, lg, nb], 
        table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'],
        k=5,
        target_attr='match', 
        metric_to_select_matcher='precision', 
        random_state=0)
result['cv_stats']

### Debugging matcher

We observe that the best matcher is not maximizing F1. We debug the matcher to see what might be wrong.
To do this, first we split the feature vectors into train and test.

In [ ]:
#  Split feature vectors into train and test
UV = em.split_train_test(H, train_proportion=0.5)
U = UV['train']
V = UV['test']

Next, we debug the matcher using GUI. For the purposes of this guide, we use random forest matcher for debugging purposes.

In [ ]:
# Debug rf using GUI
# em.vis_debug_rf(rf, U, V, 
#         exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'],
#         target_attr='match')

##  Evaluating the matching output

From the GUI, we observe that phone numbers seem to be an important attribute, but they are in different format. Current features does not capture and adding a feature incorporating this difference in format can potentially improve 
the F1 numbers.

Now, we repeat extracting feature vectors (this time with updated feature table), imputing table and selecting the best matcher again using cross-validation.

In [ ]:
H = em.extract_feature_vecs(I, feature_table=feature_table, attrs_after='match', show_progress=False)
H.fillna(0, inplace=True)

In [ ]:
# Select the best ML matcher using CV
result = em.select_matcher(
        [dt, rf, svm, ln, lg, nb], 
        table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'],
        k=5,
        target_attr='match', 
        metric_to_select_matcher='precision', 
        random_state=0)
result['cv_stats']

Evaluating the matching outputs for the evaluation set typically involves the following four steps:
1. Converting the evaluation set to feature vectors
2. Training matcher using the feature vectors extracted from the development set
3. Predicting the evaluation set using the trained matcher
4. Evaluating the predicted matches

### Converting the evaluation set to  feature vectors

As before, we convert to the feature vectors (using the feature table and the evaluation set)

In [ ]:
# Convert J into a set of feature vectors using feature table
L = em.extract_feature_vecs(
        J, 
        feature_table=feature_table,
        attrs_after='match', 
        show_progress=False)
L.fillna(0, inplace=True)
L.head()

### Training the selected matcher

Now, we train the matcher using all of the feature vectors from the development set. For the purposes of this guide we use random forest as the selected matcher.

In [ ]:
# Train using feature vectors from I 
rf.fit(table=H, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'], 
       target_attr='match')

### Predicting the matches

Next, we predict the matches for the evaluation set (using the feature vectors extracted from it).

In [ ]:
# Predict on L 
predictions = rf.predict(
        table=L, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'], 
        append=True, 
        target_attr='predicted', 
        inplace=False)

### Evaluating the predictions

Finally, we evaluate the accuracy of predicted outputs

In [ ]:
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'match', 'predicted')
em.print_eval_summary(eval_result)